In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid

from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.utils import check_random_state
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

from sklearn.ensemble import VotingClassifier
from mlxtend.classifier import StackingClassifier 
from sklearn import model_selection  
from sklearn.ensemble import BaggingClassifier


import warnings
warnings.filterwarnings('ignore')

## 特征工程

In [3]:
train_data = pd.read_csv('df_training.csv')
test_data = pd.read_csv('df_test.csv')

In [4]:
#TLSum TLMaxSum 这两个值差异化比较大

In [5]:
#train_data = train_data[train_data.TLSum < 90000]
#train_data = train_data[train_data.TLMaxSum<120000]

In [6]:
#train_data = train_data.dropna()
train_data = train_data.fillna(train_data.median())
test_data = test_data.fillna(test_data.median())

In [7]:
X = train_data.iloc[:,1:-1]
y = train_data.iloc[:,-1]
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y,test_size=0.1,random_state=1)

# 训练样本归一化
std_x = MinMaxScaler()
x_train = std_x.fit_transform(x_train)
x_test = std_x.transform(x_test)

# 测试样本归一化
std_y  = MinMaxScaler()
y_train = std_y.fit_transform(y_train.values.reshape(-1,1))
y_test = std_y.transform(y_test.values.reshape(-1,1))


# 所有训练样本的归一化
std_x_all = MinMaxScaler()
x_train_all = std_x_all.fit_transform(X)

std_y_all = MinMaxScaler()
y_train_all = std_y_all.fit_transform(y.values.reshape(-1,1))

# 预测值处理
std_x_test = MinMaxScaler()
x_test_data = std_x_test.fit_transform(test_data.iloc[:,1:-1])

## 训练

In [8]:
param_test1 = {'n_estimators':list(range(132,133,1))}
param_test2 = {'max_depth':list(range(9,13,1)), 'min_samples_split':list(range(78,82,1))}
param_test3 = { 'min_samples_leaf':range(18,24,2)}
gsearch1 = GridSearchCV(estimator = RandomForestClassifier(n_estimators=132,max_depth=11,min_samples_split=80,min_samples_leaf=20,
                                                           class_weight={0: 1, 1: 5},max_features='sqrt',oob_score=True ,random_state=10), 
                       param_grid = param_test2, scoring='f1',cv=5)
gsearch1.fit(x_train_all,y_train_all)

#scores = model_selection.cross_val_score(gsearch1, x_train_all, y_train_all, cv=5, scoring='f1')
#print("f1: %0.2f [%s]" % (scores.mean(), '得分'))
gsearch1.best_params_, gsearch1.best_score_ 


({'max_depth': 10, 'min_samples_split': 81}, 0.4343291959941716)

In [62]:
RF = RandomForestClassifier(n_estimators=138,max_depth=10,
                       min_samples_split=84,min_samples_leaf=20,class_weight={0: 1, 1: 4.1},
                       max_features='sqrt',oob_score=True ,random_state=10)
scores = model_selection.cross_val_score(RF, x_train_all, y_train_all, cv=10, scoring='accuracy')
print(scores.mean(), '随机森林accuracy')
scores = model_selection.cross_val_score(RF, x_train_all, y_train_all, cv=10, scoring='recall')
print(scores.mean(), '随机森林recall')
scores = model_selection.cross_val_score(RF, x_train_all, y_train_all, cv=10, scoring='f1')
print(scores.mean(), '随机森林f1')

scores

0.7566666666666666 随机森林accuracy
0.5866666666666667 随机森林recall
0.4449238053464687 随机森林f1


array([0.37037037, 0.46666667, 0.38961039, 0.5       , 0.475     ,
       0.40506329, 0.49350649, 0.48351648, 0.45454545, 0.4109589 ])

In [81]:
RF = RandomForestClassifier(n_estimators=138,max_depth=10,
                       min_samples_split=84,min_samples_leaf=20,class_weight={0: 1, 1: 5},
                       max_features='sqrt',oob_score=True ,random_state=10) 
#scores = model_selection.cross_val_score(RF, x_train_all, y_train_all, cv=10, scoring='f1')
#print(i,scores.mean())

In [82]:
RF.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 5},
            criterion='gini', max_depth=10, max_features='sqrt',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=20,
            min_samples_split=84, min_weight_fraction_leaf=0.0,
            n_estimators=138, n_jobs=None, oob_score=True, random_state=10,
            verbose=0, warm_start=False)

In [83]:
y_predict = RF.predict(x_test)

## 评价

In [84]:
metrics.confusion_matrix(y_test,y_predict)

array([[101,  38],
       [ 14,  27]], dtype=int64)

In [85]:
metrics.confusion_matrix(y_test,y_predict)
a = metrics.recall_score(y_test,y_predict)
print(a)
b = metrics.precision_score(y_test,y_predict)
print(b)
f1 = 2*a*b/(a + b)
print(f1)

0.6585365853658537
0.4153846153846154
0.509433962264151


In [97]:
def estimate(estimator, name='estimator'):
    auc = cross_val_score(estimator, x_train_all,y_train_all, scoring='roc_auc', cv=3).mean()
    precision = cross_val_score(estimator, x_train_all,y_train_all, scoring='precision', cv=3).mean()
    f1 = cross_val_score(estimator,x_train_all,y_train_all, scoring='f1', cv=3).mean()
    recall = cross_val_score(estimator, x_train_all,y_train_all, scoring='recall', cv=3).mean()

    print('auc',auc)
    print('precision',precision)    
    print('recall',recall)    
    print('f1',f1)        

In [99]:
from sklearn.ensemble import VotingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import cross_val_score


v1 = XGBClassifier( learning_rate =0.11, n_estimators=39, max_depth=4,
                     min_child_weight=3, gamma=0, subsample=0.8, colsample_bytree=0.8,reg_alpha=1.505,
                     objective= 'binary:logistic', nthread=4, scale_pos_weight=5,seed=27)

v2 = RandomForestClassifier(n_estimators=132,max_depth=11,min_samples_split=80,min_samples_leaf=20,
                            class_weight={0: 1, 1: 5},max_features='sqrt',oob_score=True ,random_state=10)


estimators = []
# estimators.append(('RidgeClassifier', RidgeClassifier()))
estimators.append(('LogisticRegression', LogisticRegression(solver='lbfgs',class_weight='balanced')))
estimators.append(('XGBClassifier', v1))
estimators.append(('AdaBoostClassifier', v2))
# estimators.append(('RandomForestClassifier', RandomForestClassifier()))

#voting: auc:0.794587, recall:0.000642, accuracy:0.944433

voting = VotingClassifier(estimators = estimators, voting='soft') 

from sklearn.ensemble import BaggingClassifier
bagging_clf = BaggingClassifier(voting,n_estimators=50)
# 计算交叉验证的准确率
#scores = cross_val_score(bagging_clf, train_data, train_target, cv=3)
#print(scores.mean()) 


#bagging_clf.fit(x_train,y_train)
#y_predict = bagging_clf.predict(x_test)
estimate(bagging_clf)

auc 0.7674866666666667
precision 0.35499283325370284
recall 0.5666666666666668
f1 0.429673125856332


In [94]:
metrics.confusion_matrix(y_test,y_predict)
a = metrics.recall_score(y_test,y_predict)
print(a)
b = metrics.precision_score(y_test,y_predict)
print(b)
f1 = 2*a*b/(a + b)
print(f1)

0.5609756097560976
0.40350877192982454
0.46938775510204084


In [18]:
#保存预测结果
y_predict = gsearch1.predict(x_test_data)
df = pd.DataFrame(columns=['ID','Predicted_Results'])
df['ID'] = test_data.iloc[:,0]
df['Predicted_Results'] = y_predict
df.to_csv('output/result_random.csv',index=False)

In [19]:
a = pd.read_csv('output/result_0.5.csv')

In [23]:
a.columns = ['ID','RS']

In [34]:
mer = pd.merge(a,df,on = 'ID')
mer.head()

,ID,RS,Predicted_Results
0,104191,0,0.0
1,43988,1,1.0
2,3390,1,1.0
3,37343,0,0.0
4,56827,0,0.0


In [39]:
dif = mer[mer.RS != mer.Predicted_Results]